# Đồ án cuối kì
Họ tên: Nguyễn Thanh Mẫn

MSSV: 1712586

---

## Import

In [1]:
import numpy
import pandas
from collections import Counter
import matplotlib.pyplot as plt

## Lấy dữ liệu

- Dữ liệu được sử dụng trong đồ án là thống kê về số lượng vụ tự sát trong các thành phố ở mỗi năm, được phân chia thành từng nhóm giới tính và độ tuổi khác nhau, kèm theo dân số của từng nhóm và một số thông tin về sự phát triển của thành phố đó trong năm. Nguồn của dữ liệu được lấy tại [đây](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016?select=master.csv).

- License ở [đây](https://www.worldbank.org/en/about/legal/terms-of-use-for-datasets).

- Dữ liệu được thu thập từ 4 nguồn khác nhau:

    - [UNDP's Human development index (2018)](http://hdr.undp.org/en/indicators/137506)

    - [World Bank's World development indicators: GDP (current US$) by country: 1985 to 2016 (2018)](http://databank.worldbank.org/data/source/world-development-indicators#)

    - [Szamil's Suicide in the Twenty-First Century dataset (2017)](https://www.kaggle.com/szamil/suicide-in-the-twenty-first-century/notebook)

    - [World Health Organization's Suicide prevention](http://www.who.int/mental_health/suicide-prevention/en/)

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [3]:
num_rows, num_cols = df.shape
print("Dataset has {0} rows and {1} cols".format(num_rows, num_cols))

Dataset has 27820 rows and 12 cols


### Mỗi dòng và mỗi cột có ý nghĩa gì?

Mô tả về các cột của file "who_suicide_statistics.csv":
- **country**: thành phố của nước Ma-rốc.
- **year**: năm từ 1990 đến 2017.
- **sex**: giới tính, gồm nam (male) và nữ (female).
- **age**: nhóm tuổi.
- **suicides_no**: số lượng vụ tự sát.
- **population**: dân số thuộc nhóm tuổi và thành phố đó.
Mỗi dòng là số liệu thống kê các vụ tự sát trong mỗi thành phố, nhóm tuổi và năm.
- **suicides/100k pop**: tỉ lệ tự sát trên 100,000 người.
- **country-year**: ghép giữa 2 cột **country** và **year** (cột này có vẻ không cần thiết nên sẽ lược bỏ trong bước tiền xử lí).
- **HDI for year**: chỉ số phát triển con người trong năm của thành phố đó ([Chỉ số phát triển con người là gì?](https://vi.wikipedia.org/wiki/Ch%E1%BB%89_s%E1%BB%91_ph%C3%A1t_tri%E1%BB%83n_con_ng%C6%B0%E1%BB%9Di)).
- **gdp_for_year**: chỉ số GDP trong năm của thành phố đó.
- **gdp_per_capita**: chỉ số GDP/ người.
- **generation**: thế hệ thuộc nhóm tuổi tương ứng.

Mỗi dòng là thống kê các vụ tự sát và thông tin về sự phát triển của mỗi nhóm tuổi sống trong một thành phố trong mỗi năm.

### Mỗi cột có dtype là gì?

In [4]:
col_dtypes = df.dtypes
col_dtypes

country                object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
HDI for year          float64
 gdp_for_year ($)      object
gdp_per_capita ($)      int64
generation             object
dtype: object

## Tiền xử lý

### Chỉnh lại tên của các cột cho hợp lý

In [5]:
df.rename(columns={
    df.columns[2]: 'gender',
    df.columns[4]: 'nsuicides',
    df.columns[6]: 'crudeRate',
    df.columns[8]: 'HDIyear',
    df.columns[9]: 'GDPyear',
    df.columns[10]: 'GDPcap'
},
          inplace=True)

### Bỏ cột country-year

In [6]:
df.drop(columns='country-year', inplace=True)

### Bỏ hậu tố ' years' trong các phần tử của cột "age"

In [7]:
df.age = df.age.str.replace(' years', '')

### Sửa các phần tử "5-14" trong cột "age" thành "05-14" để dễ sort

In [8]:
df.loc[df.age == '5-14', 'age'] = '05-14'

### Chỉnh dtype của cột "GDPyear" thành dạng numeric

Cột "GDPyear" đang có dtype là object, để chuyển thành dạng numeric trước tiên ta cần loại bỏ các dấu phẩy của tất cả các hàng trong cột

In [9]:
df.GDPyear = df.GDPyear.str.replace(',', '')

Sau khi bỏ hết dấu phẩy, dùng `pd.to_numeric` để chuyển dtype thành dạng numeric

In [10]:
df.GDPyear = pd.to_numeric(df.GDPyear)
df.GDPyear.dtype

dtype('int64')

## Quay lại bước khám phá dữ liệu

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     27820 non-null  object 
 1   year        27820 non-null  int64  
 2   gender      27820 non-null  object 
 3   age         27820 non-null  object 
 4   nsuicides   27820 non-null  int64  
 5   population  27820 non-null  int64  
 6   crudeRate   27820 non-null  float64
 7   HDIyear     8364 non-null   float64
 8   GDPyear     27820 non-null  int64  
 9   GDPcap      27820 non-null  int64  
 10  generation  27820 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 2.3+ MB


### Các cột dạng numeric có miền giá trị như thế nào?

Hiện đang có $6$ cột có dạng numeric. Miền giá trị của từng cột sẽ được lưu vào dataframe `col_ranges_df` có $2$ dòng là "min", "max" và $7$ cột là  "nsuicides", "population", "crudeRate", "HDIyear", "GDPyear", "GDPcap".

In [12]:
numeric_cols = [
    "nsuicides", "population", "crudeRate", "HDIyear", "GDPyear",
    "GDPcap"
]
col_ranges_df = pd.DataFrame()
col_ranges_df['min'] = df[numeric_cols].min()
col_ranges_df['max'] = df[numeric_cols].max()
col_ranges_df = col_ranges_df.T.astype(df[numeric_cols].dtypes.to_dict())
col_ranges_df

,nsuicides,population,crudeRate,HDIyear,GDPyear,GDPcap
min,0,278,0.00,0.483,46919625,251
max,22338,43805214,224.97,0.944,18120714000000,126352


### Các cột dạng categorical có các giá trị nào? 

Các cột dạng categorical: "year", "country", "gender", "age", "generation". Các giá trị của cột sẽ được lưu vào dataframe `cat_col_ranges_df` có $2$ dòng là "NumUniqueVals", "ThreeFirstVals và $5$ cột "year", "country", "gender", "age", "generation".

In [13]:
cat_cols = ["year", "country", "gender", "age", "generation"]
cat_col_ranges_df = pd.DataFrame(columns=cat_cols)
cat_col_ranges_df.loc["NumUniqueVals"] = [
    df[col].nunique() for col in cat_col_ranges_df.columns
]
cat_col_ranges_df.loc["SomeFirstVals"] = [
    np.sort(df[col].unique())[:3].tolist()
    for col in cat_col_ranges_df.columns
]
cat_col_ranges_df

,year,country,gender,age,generation
NumUniqueVals,32,101,2,6,6
SomeFirstVals,"[1985, 1986, 1987]","[Albania, Antigua and Barbuda, Argentina]","[female, male]","[05-14, 15-24, 25-34]","[Boomers, G.I. Generation, Generation X]"


### Mỗi cột có bao nhiêu giá trị thiếu?

In [15]:
col_num_nans = df.isnull().sum()
col_num_nans

country           0
year              0
gender            0
age               0
nsuicides         0
population        0
crudeRate         0
HDIyear       19456
GDPyear           0
GDPcap            0
generation        0
dtype: int64

---

## Đưa ra các câu hỏi cần trả lời